# Example: Measure Indexing and Retrieval Effectiveness of Terrier, Anserini, and PISA

## Imports

In [ ]:
from itertools import islice
from json import loads
from pathlib import Path
from shutil import rmtree

from ir_measures import nDCG
from pandas import DataFrame, concat
from pyterrier import Indexer, IterDictIndexer, IndexRef, Experiment
from pyterrier_anserini import AnseriniIndexer, AnseriniIndex, AnseriniRetriever
from pyterrier.datasets import get_dataset
from pyterrier.terrier import Retriever
from pyterrier_pisa import PisaIndexer, PisaIndex, PisaRetrieve
from seaborn import FacetGrid, lineplot

from tirex_tracker import tracking, TrackingHandle, Measure
from tirex_tracker.pyterrier import TrackedTransformer

## Data

Load the dataset from `ir_datasets`.

In [ ]:
dataset = get_dataset("irds:antique/test")

Show some documents.

In [ ]:
list(islice(dataset.get_corpus_iter(), 5))

Show the topics.

In [ ]:
dataset.get_topics()

Show the qrels.

In [ ]:
dataset.get_qrels()

## Indexing

We will consume the document iterator once to warm the document cache, so that the indexers are not slowed down by document downloads.

In [ ]:
for _ in dataset.get_corpus_iter():
    pass

### Terrier Indexing

In [ ]:
terrier_index_path = Path("./index-terrier").resolve()
print(str(terrier_index_path))
if terrier_index_path.exists():
    rmtree(terrier_index_path)
terrier_indexer = IterDictIndexer(str(terrier_index_path))

with tracking() as terrier_indexing_tracking_results:
    terrier_index: IndexRef = terrier_indexer.index(dataset.get_corpus_iter()) # type: ignore

### Anserini Indexing

In [ ]:
anserini_index_path = Path("./index-anserini").resolve()
if anserini_index_path.exists():
    rmtree(anserini_index_path)
anserini_indexer: Indexer = AnseriniIndexer(str(anserini_index_path))

with tracking() as anserini_indexing_tracking_results:
    anserini_index: AnseriniIndex = anserini_indexer.index(dataset.get_corpus_iter())

### PISA Indexing

In [ ]:
pisa_index_path = Path("./index-pisa").resolve()
if pisa_index_path.exists():
    rmtree(pisa_index_path)
pisa_indexer = PisaIndexer(str(pisa_index_path))
with tracking() as pisa_indexing_tracking_results:
    pisa_index: PisaIndex = pisa_indexer.index(dataset.get_corpus_iter())

### Results

In [ ]:
def tracking_results_to_df(
    tracking_results: TrackingHandle, measure: Measure
) -> DataFrame:
    measure_results = tracking_results[measure]
    print(f"Tracking results for {measure}: {measure_results}")
    measure_value = loads(measure_results.value)
    print(f"Tracking results for {measure}: {measure_value}")
    return DataFrame()

In [ ]:
df_indexing = concat([
    tracking_results_to_df(tracking_results, measure)
    for tracking_results in [
        terrier_indexing_tracking_results,
        anserini_indexing_tracking_results,
        pisa_indexing_tracking_results,
    ]
    for measure in [
        Measure.CPU_USED_PROCESS_PERCENT,
        Measure.CPU_USED_SYSTEM_PERCENT,
        Measure.RAM_USED_PROCESS_KB,
        Measure.RAM_USED_SYSTEM_MB,
    ]
])

## Retrieval

Create the BM25 retrievers for Terrier, Anserini, and PISA. Then wrap the retrievers with `TrackedTransformer` to track their execution.

In [ ]:
terrier_bm25 = TrackedTransformer(Retriever(terrier_index, wmodel="BM25"))
anserini_bm25: AnseriniRetriever = TrackedTransformer(anserini_index.bm25())
pisa_bm25: PisaRetrieve = TrackedTransformer(pisa_index.bm25())

## Experiment

In [ ]:
results = Experiment(
    retr_systems=[
        terrier_bm25,
        anserini_bm25,
        pisa_bm25,
    ],
    topics=dataset.get_topics()[:3],
    qrels=dataset.get_qrels(),
    eval_metrics=[nDCG @ 10, nDCG @ 5],
    names=[
        "Terrier BM25",
        "Anserini BM25",
        "PISA BM25",
    ],
    perquery=True,
)
results

In [ ]:
def tracking_results_to_df(tracking_results: TrackingHandle):
    return DataFrame(
        {
            "name": tracking_results.name,
            "time": tracking_results.time,
            "type": tracking_results.type,
            "value": tracking_results.value,
        }
    )